# 1. Imports

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

---
# 2. Leitura

In [0]:
schema = StructType([
    StructField("product_id",                  StringType(), False),
    StructField("product_category_name",       StringType(),  True),
    StructField("product_name_lenght",        IntegerType(),  True),
    StructField("product_description_lenght", IntegerType(),  True),
    StructField("product_weight_g",           IntegerType(),  True),
    StructField("product_length_cm",          IntegerType(),  True),
    StructField("product_height_cm",          IntegerType(),  True),
    StructField("product_width_cm",           IntegerType(),  True),
])


Como podemos ver acima o schema foi definido como `False` para a coluna `product_id`, assim garantindo que o valor dela não está nulo por se tratarem de colunas chave para identificação e relação com as demais.

In [0]:
products_file_path = [file.path for file in dbutils.fs.ls('dbfs/FileStore/Datum/KaggleOlistBrData/bronze') if '_products_' in str(file.name).lower()]

if len(products_file_path) == 0:
    # Caso não tenha nenhum arquivo para leitura retornará um erro
    raise ValueError('Sem arquivo products no dir "dbfs:/FileStore/Datum/KaggleOlistBrData/bronze/"')

if len(products_file_path) == 1:
    df_products = spark.read.csv(products_file_path[0], header=True, schema=schema)

else:
    # Se tivermos mais de um arquivo identificado para a tabela 'S_D_K_CUSTOMERS' não será processado e é necesária uma revisão
    raise ValueError('Mais de um arquivo products no dir "dbfs:/FileStore/Datum/KaggleOlistBrData/bronze/"')

---
# 4. Delta Lake

Utilizando `format("delta")` para aproveitar as vantagens da estrutura.

In [0]:
%sql

USE CATALOG datum

In [0]:
%sql

USE DATABASE silver

In [0]:
%sql

CREATE TABLE IF NOT EXISTS olist_products
(
  product_id                  STRING NOT NULL,
  product_category_name       STRING,
  product_name_lenght        INTEGER,
  product_description_lenght INTEGER,
  product_weight_g           INTEGER,
  product_length_cm          INTEGER,
  product_height_cm          INTEGER,
  product_width_cm           INTEGER
)
USING DELTA
LOCATION 'abfss://unity-datum@datumunity.dfs.core.windows.net/silver/olist_products'

In [0]:
if df_products.count() != 0 and df_products is not None:
    df_products.write.format('delta').mode("overwrite").save('abfss://unity-datum@datumunity.dfs.core.windows.net/silver/olist_products')

In [0]:
del df_products